In [1]:
from pytube import YouTube
from tqdm import tqdm
import cv2
from matplotlib import pyplot as plt
import pytesseract
import numpy as np
from PIL import Image
import copy
import os

def time_to_sec(time):
    minute, sec = time[0], time[1]
    return minute * 60 + sec

In [2]:
# Video to process

game_name = 'cg_opt_w6d2_s2019'
highlights_yt = 'https://www.youtube.com/watch?v=OQtqkc1DwT8'
game_yt = 'https://www.youtube.com/watch?v=ESv2ox7mtBQ'
game_start_in_video_time = time_to_sec([9, 48]) - 8

# game_name = '100t_fox_w6d2_s2019'
# highlights_yt = 'https://www.youtube.com/watch?v=29ixaSx1GYg'
# game_yt = 'https://www.youtube.com/watch?v=4fbDX5A14nk'
# game_start_in_video_time = time_to_sec([7, 46]) - 8

# game_name = 'opt_100t_w6d1_s2019'
# highlights_yt = 'https://www.youtube.com/watch?v=3R-5ufkDGuk'
# game_yt = 'https://www.youtube.com/watch?v=BYtrBlcV9eE'
# game_start_in_video_time = time_to_sec([25, 35]) - 8

# game_name = 'fox_c9_w6d2_s2019'
# highlights_yt = 'https://www.youtube.com/watch?v=37kLNEuSYVc'
# game_yt = 'https://www.youtube.com/watch?v=IId_rKGBsGE'
# game_start_in_video_time = time_to_sec([10, 12]) - 8

# game_name = 'fly_tl_w6d2_s2019'
# highlights_yt = 'https://www.youtube.com/watch?v=EIuoci7_hGA'
# game_yt = 'https://www.youtube.com/watch?v=uGuY_-kCVS0'
# game_start_in_video_time = time_to_sec([16, 50]) - 8

# game_name = 'c9_tsm_w6d2_s2019'
# highlights_yt = 'https://www.youtube.com/watch?v=HF6wnXp3rsE'
# game_yt = 'https://www.youtube.com/watch?v=GlqyJDKV6eg'
# game_start_in_video_time = time_to_sec([14, 20]) - 8

# game_name = 'clg_ggs_w6d2_s2019'
# highlights_yt = 'https://www.youtube.com/watch?v=YRAgGKrPVLc'
# game_yt = 'https://www.youtube.com/watch?v=_FGBtmfXzj0'
# game_start_in_video_time = time_to_sec([7, 50]) - 8

In [3]:
# Add folders
folders = ['frames', 'source', 'source/' + game_name, 'frames/' + game_name]
for f in folders:
    if not os.path.exists(f):
        os.makedirs(f)

In [4]:
# highlights
yt = YouTube(highlights_yt)

stream = yt.streams.filter(progressive=True, file_extension='mp4').order_by('resolution').desc().first()

pbar = tqdm(total=stream.filesize)
def progress_fn(self, chunk, *_):
    pbar.update(len(chunk))
yt.register_on_progress_callback(progress_fn)    

stream.download(output_path="./source/" + game_name, filename="source_high_res_highlights") 

 99%|█████████▉| 97959936/98723567 [00:22<00:00, 3992195.59it/s]

'./source/cg_opt_w6d2_s2019/source_high_res_highlights.mp4'

In [5]:
# video
yt = YouTube(game_yt)

stream = yt.streams.filter(progressive=True, file_extension='mp4').order_by('resolution').desc().first()

pbar = tqdm(total=stream.filesize)

yt.register_on_progress_callback(progress_fn)    

stream.download(output_path="./source/" + game_name, filename="src") 


 76%|███████▌  | 490749952/645206144 [00:22<00:07, 21121933.63it/s]


 99%|█████████▉| 641966080/645206144 [00:29<00:00, 18296072.28it/s]

'./source/cg_opt_w6d2_s2019/src.mp4'

In [6]:
fps = stream.fps

In [7]:
# Crop image to get timestamp

def get_num_from_img(crop_img, blur_size=None):

    crop_img = cv2.cvtColor(crop_img, cv2.COLOR_BGR2GRAY)
    flip_img = 255 - crop_img

    # ret, thresh = cv2.threshold(flip_img, 127, 255, cv2.THRESH_BINARY) 
    ret, thresh = cv2.threshold(flip_img, 170, 255, cv2.THRESH_BINARY) 

    upscale_img = cv2.resize(thresh, (thresh.shape[1] * 20, thresh.shape[0] * 20), interpolation = cv2.INTER_AREA) 
    
    if not blur_size:
        blur_size = 25
        
    blur = cv2.blur(upscale_img,(blur_size,blur_size))
        
    
    pil_img = Image.fromarray(blur)
    text = pytesseract.image_to_string(pil_img,  config='outputbase digits')
        
    kernel_sharpening = np.array([[-1,-1,-1], 
                                  [-1, 9,-1],
                                  [-1,-1,-1]])
    
    sharpened = cv2.filter2D(blur, -1, kernel_sharpening)
    pil_img = Image.fromarray(sharpened)
    if text == "":
        text = pytesseract.image_to_string(pil_img,  config='outputbase digits')

    
    pil_img.save("test.jpg", "JPEG")
    
#     if (text == "" and blur_size == None):
# #     0.8571428571428571
#         return (get_num_from_img(crop_img. blur_size + 10))
    
#     if text != "":
#         plt.figure()
#         plt.subplot(161),plt.imshow(crop_img),plt.title('Original')
#         plt.subplot(162),plt.imshow(flip_img),plt.title('Flipped')
#         plt.subplot(163),plt.imshow(thresh),plt.title('Threshold')
#         plt.subplot(164),plt.imshow(upscale_img),plt.title('Upscaled')
#         plt.subplot(165),plt.imshow(blur),plt.title('Blur ' + text)
#         plt.subplot(166),plt.imshow(sharpened),plt.title('Sharpened ' + text)
    return text

def get_timestamp_from_img(img):
    height, width, chan = img.shape

    crop_img = img[int(height * 0.068): int(height * 0.1), int(width * 0.485) : int(width * 0.52)]
    crop_img_left = img[int(height * 0.07): int(height * 0.095), int(width * 0.49) : int(width * 0.503)]
    crop_img_left = np.pad(crop_img_left, [(5, 5), (5, 5), (0,0)], mode='constant')
    crop_img_right = img[int(height * 0.07): int(height * 0.09), int(width * 0.504) : int(width * 0.515)]
    crop_img_right = np.pad(crop_img_right, [(5, 5), (5, 5), (0,0)], mode='constant')
    
    left = get_num_from_img(crop_img_left)
    try:
        left = int(left)
    except:
        left = ""
    right = get_num_from_img(crop_img_right)
    try:
        right = int(right)
    except:
        right = ""
    
    return (left, right)
        

In [8]:
# Get the raw times
times = []
vidcap = cv2.VideoCapture('source/' + game_name + '/source_high_res_highlights.mp4')
count = 0
success = True
while success:
    success,image = vidcap.read()
    
    if count % 40 == 0:
        (left, right) = get_timestamp_from_img(image)
        times.append((left, right))
        print((left, right))

    count += 1

print(times)


('', '')
('', '')
('', '')
('', '')
('', '')
('', '')
('', '')
('', '')
('', '')
('', '')
('', '')
('', '')
(9, '')
(9, '')
(9, '')



100%|██████████| 645206144/645206144 [00:45<00:00, 18296072.28it/s]

('', '')
(6, 31)
(6, 32)
(6, 34)
(6, 35)
(6, 37)
(6, 38)
(6, 39)
(6, 41)
(7, 29)
(7, 30)
(7, 31)
(7, 33)
(7, 34)
(7, 35)
(7, 37)
(7, 38)
(7, 40)
(7, 41)
(7, 42)
(7, '')
(7, 45)
(7, 47)
(7, 48)
(7, 49)
(7, 50)
(9, 54)
(9, 55)
(9, 57)
(9, 58)
(9, 59)
(10, 0)
('', 1)
('', 3)
('', 4)
(10, 5)
('', '')
('', '')
('', 46)
(10, 47)
(10, 49)
('', 50)
(10, 51)
(10, 53)
(10, 54)
(10, 55)
('', 57)
('', 58)
(11, 0)
(11, 1)
('', 2)
(13, 16)
(13, 18)
('', 19)
(13, 20)
(13, 22)
(13, 23)
(13, 24)
(13, 26)
(13, 27)
(13, 29)
(13, 30)
(13, 31)
(13, 33)
(13, 33)
(13, 35)
(13, 36)
(13, 37)
(13, 39)
(13, 40)
(13, 2)
(13, 43)
(13, 44)
(13, 46)
(13, 47)
(13, 49)
(13, 50)
('', 51)
(17, 51)
(17, 52)
(17, 54)
(17, 55)
(17, 56)
(17, 58)
(17, 59)
('', 1)
('', 2)
('', 3)
('', 5)
('', 6)
('', 7)
('', 9)
('', 10)
('', 12)
('', 13)
('', 14)
('', 15)
('', 16)
('', 18)
(18, 19)
('', 20)
(18, 22)
(18, 23)
('', 25)
(18, 26)
('', '')
('', '')
('', '')
('', '')
('', '')
(22, 20)
(22, 21)
(22, 23)
(22, 24)
(22, 26)
(22, 27)
(2

In [9]:
# Filter the first empty times (from non highlight footage)
saw_non_empty = False
times_first_non_empty = []
for time in times:
    if time[0] != "" and time[1] != "":
        saw_non_empty = True
    if time[0] == "" and time[1] == "":
        continue
    if saw_non_empty:
        times_first_non_empty.append(time)

last_time = None
# Guess the empty times
times_non_empty_temp = []
for time in times_first_non_empty:
    
    left = time[0]
    right = time[1]
    
    if last_time == None:
        last_time = [time[0], time[1]]
    else:
        # Going down in minutes, skip
        if left != "" and left < last_time[0]:
            continue
        if left != "":
            last_time[0] = left
        # New time but need to incr the minute
        if right != "" and right < last_time[1] and right < 5:
            last_time[0] += 1
        if right != "":
            last_time[1] = right
            
    times_non_empty_temp.append(copy.deepcopy(last_time))
    
# Filter to make sure none are empty
times_non_empty = []
for time in times_non_empty_temp:
    if (time[0] != "" and time[1] != ""):
        times_non_empty.append(time)
   
print(times_non_empty)


[[6, 31], [6, 32], [6, 34], [6, 35], [6, 37], [6, 38], [6, 39], [6, 41], [7, 29], [7, 30], [7, 31], [7, 33], [7, 34], [7, 35], [7, 37], [7, 38], [7, 40], [7, 41], [7, 42], [7, 42], [7, 45], [7, 47], [7, 48], [7, 49], [7, 50], [9, 54], [9, 55], [9, 57], [9, 58], [9, 59], [11, 0], [11, 1], [11, 3], [11, 4], [11, 46], [11, 50], [11, 57], [11, 58], [12, 0], [12, 2], [13, 16], [13, 18], [13, 19], [13, 20], [13, 22], [13, 23], [13, 24], [13, 26], [13, 27], [13, 29], [13, 30], [13, 31], [13, 33], [13, 33], [13, 35], [13, 36], [13, 37], [13, 39], [13, 40], [14, 2], [14, 51], [17, 51], [17, 52], [17, 54], [17, 55], [17, 56], [17, 58], [17, 59], [18, 1], [18, 2], [18, 3], [18, 5], [18, 6], [18, 7], [18, 9], [18, 10], [18, 12], [18, 13], [18, 14], [18, 15], [18, 16], [18, 18], [18, 19], [18, 20], [18, 22], [18, 23], [18, 25], [18, 26], [22, 20], [22, 21], [22, 23], [22, 24], [22, 26], [22, 27], [22, 28], [22, 30], [22, 31], [22, 33], [22, 34], [22, 34], [22, 36], [22, 37], [22, 39], [22, 40], [22

In [10]:
# Remove extremes
non_extreme_times = []
for i in range(len(times_non_empty)):
    left_diff = 0
    if i-1 > 0:
        left_diff = abs(time_to_sec(times_non_empty[i]) - time_to_sec(times_non_empty[i-1]))
    right_diff = 0
    if i+1 < len(times_non_empty):
        right_diff =  abs(time_to_sec(times_non_empty[i]) - time_to_sec(times_non_empty[i+1]))
    if left_diff > 10 and right_diff > 10:
        continue
    
    non_extreme_times.append(times_non_empty[i])
print(non_extreme_times)

[[6, 31], [6, 32], [6, 34], [6, 35], [6, 37], [6, 38], [6, 39], [6, 41], [7, 29], [7, 30], [7, 31], [7, 33], [7, 34], [7, 35], [7, 37], [7, 38], [7, 40], [7, 41], [7, 42], [7, 42], [7, 45], [7, 47], [7, 48], [7, 49], [7, 50], [9, 54], [9, 55], [9, 57], [9, 58], [9, 59], [11, 0], [11, 1], [11, 3], [11, 4], [11, 46], [11, 50], [11, 57], [11, 58], [12, 0], [12, 2], [13, 16], [13, 18], [13, 19], [13, 20], [13, 22], [13, 23], [13, 24], [13, 26], [13, 27], [13, 29], [13, 30], [13, 31], [13, 33], [13, 33], [13, 35], [13, 36], [13, 37], [13, 39], [13, 40], [17, 51], [17, 52], [17, 54], [17, 55], [17, 56], [17, 58], [17, 59], [18, 1], [18, 2], [18, 3], [18, 5], [18, 6], [18, 7], [18, 9], [18, 10], [18, 12], [18, 13], [18, 14], [18, 15], [18, 16], [18, 18], [18, 19], [18, 20], [18, 22], [18, 23], [18, 25], [18, 26], [22, 20], [22, 21], [22, 23], [22, 24], [22, 26], [22, 27], [22, 28], [22, 30], [22, 31], [22, 33], [22, 34], [22, 34], [22, 36], [22, 37], [22, 39], [22, 40], [22, 41], [22, 43], [2

In [11]:
timestamps = []

start_time = None
last_time = None
for i in range(len(non_extreme_times)):
    
    time = non_extreme_times[i]
    
    # Start of the highlight
    if start_time == None:
        start_time = time
    
    # Check if this is end of highlight
    if last_time != None and time_to_sec(time) - time_to_sec(last_time) > 10 or i == len(non_extreme_times)-1:
        timestamps.append((copy.copy(start_time), copy.copy(last_time)))
        start_time = copy.copy(time)
    
    last_time = time

print(timestamps)

[([6, 31], [6, 41]), ([7, 29], [7, 50]), ([9, 54], [9, 59]), ([11, 0], [11, 4]), ([11, 46], [12, 2]), ([13, 16], [13, 40]), ([17, 51], [18, 26]), ([22, 20], [22, 51]), ([25, 38], [25, 42]), ([27, 26], [27, 36]), ([29, 25], [29, 33]), ([30, 55], [30, 59]), ([32, 1], [32, 2])]


In [12]:
# Remove small timestamps and convert to sec
filtered_timestamps = []

for timestamp in timestamps:
    if (time_to_sec(timestamp[1]) - time_to_sec(timestamp[0])) > 5:
        filtered_timestamps.append(range(time_to_sec(timestamp[0]), time_to_sec(timestamp[1])))
print(filtered_timestamps)

[range(391, 401), range(449, 470), range(706, 722), range(796, 820), range(1071, 1106), range(1340, 1371), range(1646, 1656), range(1765, 1773)]


In [13]:
game_start_in_video_time = time_to_sec([7, 50]) - 8
print(game_start_in_video_time)

vidcap = cv2.VideoCapture('source/' + game_name + '/src.mp4')

print(vidcap.get(cv2.CAP_PROP_FPS))
fps = round(vidcap.get(cv2.CAP_PROP_FPS))
print('fps', fps)
count = 0
success = True
total_frames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
total_time = total_frames / fps
while success:
    success,image = vidcap.read()    
    
    if count % fps == 0:
        game_time = (count / fps) - game_start_in_video_time

        
        is_highlight = False
        print(str(game_time) + " / " + str(total_time))
        for r in filtered_timestamps:
            if game_time in r:
                is_highlight = True
                break
            
        if is_highlight:
            cv2.imwrite('frames/' + game_name + '/frame%d-h.jpg' % (count / fps), image)

        else:
            cv2.imwrite('frames/' + game_name + '/frame%d-n.jpg' % (count / fps), image)

    count += 1


462
29.97
fps 30
-462.0 / 2646.1666666666665
-461.0 / 2646.1666666666665
-460.0 / 2646.1666666666665
-459.0 / 2646.1666666666665
-458.0 / 2646.1666666666665
-457.0 / 2646.1666666666665
-456.0 / 2646.1666666666665
-455.0 / 2646.1666666666665
-454.0 / 2646.1666666666665
-453.0 / 2646.1666666666665
-452.0 / 2646.1666666666665
-451.0 / 2646.1666666666665
-450.0 / 2646.1666666666665
-449.0 / 2646.1666666666665
-448.0 / 2646.1666666666665
-447.0 / 2646.1666666666665
-446.0 / 2646.1666666666665
-445.0 / 2646.1666666666665
-444.0 / 2646.1666666666665
-443.0 / 2646.1666666666665
-442.0 / 2646.1666666666665
-441.0 / 2646.1666666666665
-440.0 / 2646.1666666666665
-439.0 / 2646.1666666666665
-438.0 / 2646.1666666666665
-437.0 / 2646.1666666666665
-436.0 / 2646.1666666666665
-435.0 / 2646.1666666666665
-434.0 / 2646.1666666666665
-433.0 / 2646.1666666666665
-432.0 / 2646.1666666666665
-431.0 / 2646.1666666666665
-430.0 / 2646.1666666666665
-429.0 / 2646.1666666666665
-428.0 / 2646.1666666666665
-42

-169.0 / 2646.1666666666665
-168.0 / 2646.1666666666665
-167.0 / 2646.1666666666665
-166.0 / 2646.1666666666665
-165.0 / 2646.1666666666665
-164.0 / 2646.1666666666665
-163.0 / 2646.1666666666665
-162.0 / 2646.1666666666665
-161.0 / 2646.1666666666665
-160.0 / 2646.1666666666665
-159.0 / 2646.1666666666665
-158.0 / 2646.1666666666665
-157.0 / 2646.1666666666665
-156.0 / 2646.1666666666665
-155.0 / 2646.1666666666665
-154.0 / 2646.1666666666665
-153.0 / 2646.1666666666665
-152.0 / 2646.1666666666665
-151.0 / 2646.1666666666665
-150.0 / 2646.1666666666665
-149.0 / 2646.1666666666665
-148.0 / 2646.1666666666665
-147.0 / 2646.1666666666665
-146.0 / 2646.1666666666665
-145.0 / 2646.1666666666665
-144.0 / 2646.1666666666665
-143.0 / 2646.1666666666665
-142.0 / 2646.1666666666665
-141.0 / 2646.1666666666665
-140.0 / 2646.1666666666665
-139.0 / 2646.1666666666665
-138.0 / 2646.1666666666665
-137.0 / 2646.1666666666665
-136.0 / 2646.1666666666665
-135.0 / 2646.1666666666665
-134.0 / 2646.166666

138.0 / 2646.1666666666665
139.0 / 2646.1666666666665
140.0 / 2646.1666666666665
141.0 / 2646.1666666666665
142.0 / 2646.1666666666665
143.0 / 2646.1666666666665
144.0 / 2646.1666666666665
145.0 / 2646.1666666666665
146.0 / 2646.1666666666665
147.0 / 2646.1666666666665
148.0 / 2646.1666666666665
149.0 / 2646.1666666666665
150.0 / 2646.1666666666665
151.0 / 2646.1666666666665
152.0 / 2646.1666666666665
153.0 / 2646.1666666666665
154.0 / 2646.1666666666665
155.0 / 2646.1666666666665
156.0 / 2646.1666666666665
157.0 / 2646.1666666666665
158.0 / 2646.1666666666665
159.0 / 2646.1666666666665
160.0 / 2646.1666666666665
161.0 / 2646.1666666666665
162.0 / 2646.1666666666665
163.0 / 2646.1666666666665
164.0 / 2646.1666666666665
165.0 / 2646.1666666666665
166.0 / 2646.1666666666665
167.0 / 2646.1666666666665
168.0 / 2646.1666666666665
169.0 / 2646.1666666666665
170.0 / 2646.1666666666665
171.0 / 2646.1666666666665
172.0 / 2646.1666666666665
173.0 / 2646.1666666666665
174.0 / 2646.1666666666665
1

442.0 / 2646.1666666666665
443.0 / 2646.1666666666665
444.0 / 2646.1666666666665
445.0 / 2646.1666666666665
446.0 / 2646.1666666666665
447.0 / 2646.1666666666665
448.0 / 2646.1666666666665
449.0 / 2646.1666666666665
450.0 / 2646.1666666666665
451.0 / 2646.1666666666665
452.0 / 2646.1666666666665
453.0 / 2646.1666666666665
454.0 / 2646.1666666666665
455.0 / 2646.1666666666665
456.0 / 2646.1666666666665
457.0 / 2646.1666666666665
458.0 / 2646.1666666666665
459.0 / 2646.1666666666665
460.0 / 2646.1666666666665
461.0 / 2646.1666666666665
462.0 / 2646.1666666666665
463.0 / 2646.1666666666665
464.0 / 2646.1666666666665
465.0 / 2646.1666666666665
466.0 / 2646.1666666666665
467.0 / 2646.1666666666665
468.0 / 2646.1666666666665
469.0 / 2646.1666666666665
470.0 / 2646.1666666666665
471.0 / 2646.1666666666665
472.0 / 2646.1666666666665
473.0 / 2646.1666666666665
474.0 / 2646.1666666666665
475.0 / 2646.1666666666665
476.0 / 2646.1666666666665
477.0 / 2646.1666666666665
478.0 / 2646.1666666666665
4

746.0 / 2646.1666666666665
747.0 / 2646.1666666666665
748.0 / 2646.1666666666665
749.0 / 2646.1666666666665
750.0 / 2646.1666666666665
751.0 / 2646.1666666666665
752.0 / 2646.1666666666665
753.0 / 2646.1666666666665
754.0 / 2646.1666666666665
755.0 / 2646.1666666666665
756.0 / 2646.1666666666665
757.0 / 2646.1666666666665
758.0 / 2646.1666666666665
759.0 / 2646.1666666666665
760.0 / 2646.1666666666665
761.0 / 2646.1666666666665
762.0 / 2646.1666666666665
763.0 / 2646.1666666666665
764.0 / 2646.1666666666665
765.0 / 2646.1666666666665
766.0 / 2646.1666666666665
767.0 / 2646.1666666666665
768.0 / 2646.1666666666665
769.0 / 2646.1666666666665
770.0 / 2646.1666666666665
771.0 / 2646.1666666666665
772.0 / 2646.1666666666665
773.0 / 2646.1666666666665
774.0 / 2646.1666666666665
775.0 / 2646.1666666666665
776.0 / 2646.1666666666665
777.0 / 2646.1666666666665
778.0 / 2646.1666666666665
779.0 / 2646.1666666666665
780.0 / 2646.1666666666665
781.0 / 2646.1666666666665
782.0 / 2646.1666666666665
7

1048.0 / 2646.1666666666665
1049.0 / 2646.1666666666665
1050.0 / 2646.1666666666665
1051.0 / 2646.1666666666665
1052.0 / 2646.1666666666665
1053.0 / 2646.1666666666665
1054.0 / 2646.1666666666665
1055.0 / 2646.1666666666665
1056.0 / 2646.1666666666665
1057.0 / 2646.1666666666665
1058.0 / 2646.1666666666665
1059.0 / 2646.1666666666665
1060.0 / 2646.1666666666665
1061.0 / 2646.1666666666665
1062.0 / 2646.1666666666665
1063.0 / 2646.1666666666665
1064.0 / 2646.1666666666665
1065.0 / 2646.1666666666665
1066.0 / 2646.1666666666665
1067.0 / 2646.1666666666665
1068.0 / 2646.1666666666665
1069.0 / 2646.1666666666665
1070.0 / 2646.1666666666665
1071.0 / 2646.1666666666665
1072.0 / 2646.1666666666665
1073.0 / 2646.1666666666665
1074.0 / 2646.1666666666665
1075.0 / 2646.1666666666665
1076.0 / 2646.1666666666665
1077.0 / 2646.1666666666665
1078.0 / 2646.1666666666665
1079.0 / 2646.1666666666665
1080.0 / 2646.1666666666665
1081.0 / 2646.1666666666665
1082.0 / 2646.1666666666665
1083.0 / 2646.166666

1342.0 / 2646.1666666666665
1343.0 / 2646.1666666666665
1344.0 / 2646.1666666666665
1345.0 / 2646.1666666666665
1346.0 / 2646.1666666666665
1347.0 / 2646.1666666666665
1348.0 / 2646.1666666666665
1349.0 / 2646.1666666666665
1350.0 / 2646.1666666666665
1351.0 / 2646.1666666666665
1352.0 / 2646.1666666666665
1353.0 / 2646.1666666666665
1354.0 / 2646.1666666666665
1355.0 / 2646.1666666666665
1356.0 / 2646.1666666666665
1357.0 / 2646.1666666666665
1358.0 / 2646.1666666666665
1359.0 / 2646.1666666666665
1360.0 / 2646.1666666666665
1361.0 / 2646.1666666666665
1362.0 / 2646.1666666666665
1363.0 / 2646.1666666666665
1364.0 / 2646.1666666666665
1365.0 / 2646.1666666666665
1366.0 / 2646.1666666666665
1367.0 / 2646.1666666666665
1368.0 / 2646.1666666666665
1369.0 / 2646.1666666666665
1370.0 / 2646.1666666666665
1371.0 / 2646.1666666666665
1372.0 / 2646.1666666666665
1373.0 / 2646.1666666666665
1374.0 / 2646.1666666666665
1375.0 / 2646.1666666666665
1376.0 / 2646.1666666666665
1377.0 / 2646.166666

1635.0 / 2646.1666666666665
1636.0 / 2646.1666666666665
1637.0 / 2646.1666666666665
1638.0 / 2646.1666666666665
1639.0 / 2646.1666666666665
1640.0 / 2646.1666666666665
1641.0 / 2646.1666666666665
1642.0 / 2646.1666666666665
1643.0 / 2646.1666666666665
1644.0 / 2646.1666666666665
1645.0 / 2646.1666666666665
1646.0 / 2646.1666666666665
1647.0 / 2646.1666666666665
1648.0 / 2646.1666666666665
1649.0 / 2646.1666666666665
1650.0 / 2646.1666666666665
1651.0 / 2646.1666666666665
1652.0 / 2646.1666666666665
1653.0 / 2646.1666666666665
1654.0 / 2646.1666666666665
1655.0 / 2646.1666666666665
1656.0 / 2646.1666666666665
1657.0 / 2646.1666666666665
1658.0 / 2646.1666666666665
1659.0 / 2646.1666666666665
1660.0 / 2646.1666666666665
1661.0 / 2646.1666666666665
1662.0 / 2646.1666666666665
1663.0 / 2646.1666666666665
1664.0 / 2646.1666666666665
1665.0 / 2646.1666666666665
1666.0 / 2646.1666666666665
1667.0 / 2646.1666666666665
1668.0 / 2646.1666666666665
1669.0 / 2646.1666666666665
1670.0 / 2646.166666

1928.0 / 2646.1666666666665
1929.0 / 2646.1666666666665
1930.0 / 2646.1666666666665
1931.0 / 2646.1666666666665
1932.0 / 2646.1666666666665
1933.0 / 2646.1666666666665
1934.0 / 2646.1666666666665
1935.0 / 2646.1666666666665
1936.0 / 2646.1666666666665
1937.0 / 2646.1666666666665
1938.0 / 2646.1666666666665
1939.0 / 2646.1666666666665
1940.0 / 2646.1666666666665
1941.0 / 2646.1666666666665
1942.0 / 2646.1666666666665
1943.0 / 2646.1666666666665
1944.0 / 2646.1666666666665
1945.0 / 2646.1666666666665
1946.0 / 2646.1666666666665
1947.0 / 2646.1666666666665
1948.0 / 2646.1666666666665
1949.0 / 2646.1666666666665
1950.0 / 2646.1666666666665
1951.0 / 2646.1666666666665
1952.0 / 2646.1666666666665
1953.0 / 2646.1666666666665
1954.0 / 2646.1666666666665
1955.0 / 2646.1666666666665
1956.0 / 2646.1666666666665
1957.0 / 2646.1666666666665
1958.0 / 2646.1666666666665
1959.0 / 2646.1666666666665
1960.0 / 2646.1666666666665
1961.0 / 2646.1666666666665
1962.0 / 2646.1666666666665
1963.0 / 2646.166666

In [14]:
# Remove
src = 'source/' + game_name + '/src.mp4'
src_highlights = 'source/' + game_name + '/source_high_res_highlights.mp4'
os.remove(src)
os.remove(src_highlights)